# Table of Contents

<a id="libraries"></a>
### Import necessary libraries

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import itertools

## for converting images to arrays
import os, cv2

## for train/test/val split
from sklearn.model_selection import train_test_split

## for developing a neural network
import tensorflow as tf
import keras_metrics

from tensorflow import keras
from keras import backend as K
from keras.models import Sequential
from keras.layers import Activation
from keras.layers.core import Dense
from keras.optimizers import Adam
from keras.metrics import categorical_crossentropy
from keras.preprocessing.image import ImageDataGenerator ## image augmentation

## for CNN
from keras.layers import Conv2D ## Images are 2D convolutions
from keras.layers import MaxPooling2D ## Get max features for 2D image
from keras.layers import Flatten ## Convert all the features into 1D matrix
from keras.layers import Dense ## connect fully connected layer into ANN

## for classifier/model evaluation
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.metrics import roc_curve, roc_auc_score

%matplotlib inline

/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


Couldn't import dot_parser, loading of dot files will not be possible.


Using TensorFlow backend.


### Obtain the test/train/validation set

In [2]:
## Import the csv file with merged descriptions
merged_df = pd.read_csv('../merged_data',sep='\t',index_col=0)

In [3]:
merged_df.head()

,patientId,x,y,width,height,target,target_class_desc
0,0004cfab-14fd-4e49-80ba-63a80b6bddd6,NaN,NaN,NaN,NaN,0,No Lung Opacity / Not Normal
1,00313ee0-9eaa-42f4-b0ab-c148ed3241cd,NaN,NaN,NaN,NaN,0,No Lung Opacity / Not Normal
2,00322d4d-1c29-4943-afc9-b6754be640eb,NaN,NaN,NaN,NaN,0,No Lung Opacity / Not Normal
3,003d8fa0-6bf1-40ed-b54c-ac657f8495c5,NaN,NaN,NaN,NaN,0,Normal
4,00436515-870c-4b36-a041-de91049b9ab4,264.0,152.0,213.0,379.0,1,Lung Opacity


#### Merge target class descriptions together

In [4]:
## Convert all the No Lung Opacity/Not Normal classes into Normal as well
## as the goal is just to predict pneumonia or no pneumonia

merged_df['target_class_desc'][merged_df.target == 0]='Normal'

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.


In [5]:
merged_df.head(10)

,patientId,x,y,width,height,target,target_class_desc
0,0004cfab-14fd-4e49-80ba-63a80b6bddd6,NaN,NaN,NaN,NaN,0,Normal
1,00313ee0-9eaa-42f4-b0ab-c148ed3241cd,NaN,NaN,NaN,NaN,0,Normal
2,00322d4d-1c29-4943-afc9-b6754be640eb,NaN,NaN,NaN,NaN,0,Normal
3,003d8fa0-6bf1-40ed-b54c-ac657f8495c5,NaN,NaN,NaN,NaN,0,Normal
4,00436515-870c-4b36-a041-de91049b9ab4,264.0,152.0,213.0,379.0,1,Lung Opacity
5,00436515-870c-4b36-a041-de91049b9ab4,562.0,152.0,256.0,453.0,1,Lung Opacity
6,00569f44-917d-4c86-a842-81832af98c30,NaN,NaN,NaN,NaN,0,Normal
7,006cec2e-6ce2-4549-bffa-eadfcd1e9970,NaN,NaN,NaN,NaN,0,Normal
8,00704310-78a8-4b38-8475-49f4573b2dbb,323.0,577.0,160.0,104.0,1,Lung Opacity
9,00704310-78a8-4b38-8475-49f4573b2dbb,695.0,575.0,162.0,137.0,1,Lung Opacity


#### Convert images to ndarrays

###### Note:

We are converting images to array because it is more flexible to test on different algorithms when we use arrays. Having a certain file structure as required by Keras might not always be a good idea although it is easier.

In [6]:
path = os.path.join('..','datasets','png_converted_files')

In [7]:
path

'../datasets/png_converted_files'

In [9]:
def image_to_arrays():
    """
    Returns two arrays: 
        x is an array of resized images
        y is an array of labels
    """

    x = [] # images as arrays
    y = [] # labels 
    WIDTH = 128
    HEIGHT = 128

    #for image in enumerate(merged_df.patientId.head(5000)):
    for image in enumerate(merged_df.patientId):    
        
        img_name = image[1]
        image_path = path + '/' + img_name + '.png'

        # Read and resize image
        full_size_image = cv2.imread(image_path)
        gray_image = cv2.cvtColor(full_size_image, cv2.COLOR_BGR2GRAY)
        x.append(cv2.resize(gray_image, (WIDTH,HEIGHT), interpolation=cv2.INTER_CUBIC))
        
        # Labels
        index_of_image = image[0]
        target_value = merged_df.target.loc[index_of_image]
        #print(target_value)
        y.append(target_value)

    return x,y

In [10]:
X,y = image_to_arrays()

In [19]:
X[2]

array([[17, 14, 14, ..., 18, 18, 20],
       [16, 14, 14, ..., 17, 17, 18],
       [17, 14, 14, ..., 16, 17, 18],
       ...,
       [14, 13, 13, ..., 16, 19, 23],
       [19, 17, 17, ..., 20, 29, 40],
       [25, 25, 25, ..., 21, 26, 45]], dtype=uint8)

In [ ]:
# Set it up as a dataframe if you like
df = pd.DataFrame()
df["labels"]=y
df["images"]=X

In [10]:
## Saving arrays for future use
np.savez("./data_as_arrays/x_images_arrays", X)
np.savez("./data_as_arrays/y_pneumonia_labels", y)

In [6]:
## Load a numpy array (Optinal.. only use when needed)
# Load npz file containing image arrays
x_npz = np.load("./data_as_arrays/x_images_arrays_all.npz")
X = x_npz['arr_0']
# Load binary encoded labels for Pneumonia
y_npz = np.load("./data_as_arrays/y_pneumonia_labels_all.npz")
y = y_npz['arr_0']

FileNotFoundError: [Errno 2] No such file or directory: './data_as_arrays/x_images_arrays_all.npz'

#### Split the array into train/test set

In [20]:
# First split the data in two sets, 80% for training, 20% for Val/Test)
X_train, X_test_val, y_train, y_test_val = train_test_split(X,y, test_size=0.2, random_state=1, stratify=y)

In [21]:
# Second split the 20% into validation and test sets
X_test, X_val, y_test, y_val = train_test_split(X_test_val, y_test_val, test_size=0.5, random_state=1, 
                                                stratify=y_test_val)

In [22]:
print(np.array(X_train).shape)
print(np.array(X_val).shape)
print(np.array(X_test).shape)

(23191, 128, 128)
(2899, 128, 128)
(2899, 128, 128)


In [23]:
## Saving arrays for future use
np.savez_compressed("./data_as_arrays/x_images", X)
np.savez_compressed("./data_as_arrays/y_pneumonia", y)

###### Note:

We will be fitting our model on the training set and predict using the test set. But with neural models, we will be need to tune our models a lot. So, for that it is important to have a validation set as well. 

#### Normalize the dataset

In [ ]:
X_train_array = np.array(X_train)
X_test_array = np.array(X_test)
X_val_array = np.array(X_val)

In [ ]:
## Reshape the input arrays such that the values are between 0 and 1
X_train_array = X_train_array/255.0
X_test_array = X_test_array/255.0
X_val_array = X_val_array/255.0

In [ ]:
y_train = np.array(y_train)
y_test = np.array(y_test)
y_val = np.array(y_val)

#### Check the baseline

In [ ]:
merged_df.head(5000).target_class_desc.value_counts(normalize=True)

###### Note:

Before computing any model, we need to check the baseline value. We need to do the distribution of the target variable and its baseline value. Baseline score will provide us with minimum level that our model must achieve to be accepted. 

## Fitting Models

###### Note:

Now, we will compute different models and compare the performance through different evaluation metrics. The first one is a multi-layered perceptron. We will fit a simple neural network and evaluate the performance. 

#### Fit a Multi-Layer Perceptron Model

In [ ]:
## Our MLP Model
neural_model = keras.Sequential([
    keras.layers.Flatten(input_shape=(128, 128)),
    keras.layers.Dense(128, activation=tf.nn.relu), ## input layer
    keras.layers.Dense(64, activation=tf.nn.relu), ## first hidden layer
    keras.layers.Dense(32, activation=tf.nn.relu), ## second hidden layer
    keras.layers.Dense(1, activation=tf.nn.sigmoid) ## output layer
])

In [ ]:
## Compile our model together
neural_model.compile(optimizer=tf.train.AdamOptimizer(), 
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [ ]:
## print the neural model summary
neural_model.summary()

In [ ]:
## Fitting the model
y_pred = neural_model.fit(X_train_array, y_train, epochs=25)

###### Making Predictions

In [ ]:
## Evaluating the model in the test set
test_loss, test_acc = neural_model.evaluate(X_test_array, 
                                            y_test)

In [ ]:
print('Test accuracy:', test_acc)

In [ ]:
# calculate predictions
predictions = neural_model.predict(X_test_array)

In [ ]:
predictions

In [ ]:
# round predictions
rounded_predictions = [round(x[0]) for x in predictions]
print(rounded_predictions)

In [ ]:
rounded_predictions = np.array(rounded_predictions)

In [ ]:
rounded_predictions.shape

In [ ]:
y_test.shape

#### Note: 
Out of 5,000 images taken, 
1. Our baseline is 0.52
2. Our test accuracy is 0.776 
3. So, let's normalize the distribution of our target classes such that both classes are evenly distributed

#### Evaluate the multi-layered perceptron

###### Confusion matrix

In [ ]:
dict_characters = {0: 'Normal', 1: 'Pneumonia'}
def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
    plt.figure(figsize = (5,5))
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=0)
    plt.yticks(tick_marks, classes)
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, cm[i, j],
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")
    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')

In [ ]:
# compute the confusion matrix
confusion_mtx = confusion_matrix(y_test, rounded_predictions) 

In [ ]:
classes = ['Normal', 'Lung Opacity']

In [ ]:
## Plot the confusion matrix
plt.figure()
plot_confusion_matrix(confusion_mtx, classes)
plt.xticks(range(2), ['Normal', 'Pneumonia'], fontsize=16)
plt.yticks(range(2), ['Normal', 'Pneumonia'], fontsize=16)
plt.show()


###### Classification Report

In [ ]:
## Print classification report
print(classification_report(y_test, rounded_predictions, 
                            target_names = classes))

###### AUC/ROC curve

In [ ]:
# calculate AUC
auc = roc_auc_score(y_test, rounded_predictions)
print('AUC: %.3f' % auc)

In [ ]:
# calculate roc curve
fpr, tpr, thresholds = roc_curve(y_test, rounded_predictions)

In [ ]:
# plot random line in the middle
fig, ax1 = plt.subplots(1,1, figsize = (5, 5), dpi = 250)
plt.plot([0, 1], [0, 1], linestyle='--')
# plot the roc curve for the model
plt.plot(fpr, tpr, marker='.',
        label = 'MLP Model (AUC:%.3f)' % auc)
# show the plot
ax1.legend(loc = 4)
plt.title('Receiver operating characteristic')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.show()

#### Report:

A number of conclusions can be made from our result.
1. As more dataset are included, the precision is drastically decreased with MLP.
2. The baseline was 70% and there was a high class imbalance. Still, the precision was very low. 
3. Lets fix the class imbalance by image augmentation and also fit CNN.

## Fitting CNN model

###### Define static variables

In [ ]:
## Image size after resizing
img_width, img_height = 128, 128

## no of training samples
nb_train_samples = len(X_train)

## no of validation samples
nb_validation_samples = len(X_val)

## No of epochs/iterations
epochs = 10

## batch size in each epoch
batch_size = 16

##### Developing a model

In [ ]:
## Initialize the CNN
cnn_model = Sequential()

## Step 1 - Convolution (filters)
cnn_model.add(Conv2D(32, (3, 3), input_shape = (img_width,img_height,1),
                            activation = 'relu'))

## Step 2 - Pooling
cnn_model.add(MaxPooling2D(pool_size=(2,2)))

## add another Convolution (filters)
cnn_model.add(Conv2D(32, (3, 3), activation = 'relu'))


## Step 2 - Pooling (downsize to get non-overlapping areas with max values)
cnn_model.add(MaxPooling2D(pool_size=(2,2)))

## add another Convolution (filters)
cnn_model.add(Conv2D(64, (3, 3), activation = 'relu'))

## Step 2 - Pooling (downsize to get non-overlapping areas with max values)
cnn_model.add(MaxPooling2D(pool_size=(2,2)))

## Step 3 - Flattening (convert 3D feature matrix to 1D vector)
cnn_model.add(Flatten())

## Step 4 - Make fully connected layer
cnn_model.add(Dense(units = 128, activation = 'relu'))
cnn_model.add(Dense(units = 64, activation = 'relu'))
cnn_model.add(Dense(units = 32, activation = 'relu'))
cnn_model.add(Dense(units = 1, activation = 'sigmoid'))

In [ ]:
## Compiling the CNN
cnn_model.compile(optimizer='adam', loss = 'binary_crossentropy',
                  metrics = ['accuracy'])

In [ ]:
## Print the summary
cnn_model.summary()

##### Image Augmentation

###### Although, our model is prepared, we still need to fix the class imbalance. With Image Augmentation, techniques we can create new images from the existing images. Image generator in keras enables us to achieve this

In [ ]:
train_datagen = ImageDataGenerator(rescale=1. / 255, 
                                   horizontal_flip=True, 
                                   rotation_range=30,
                                  width_shift_range=0.1,
                                  height_shift_range=0.1, 
                                  shear_range=0.15,
                                  zoom_range=0.1,
                                  channel_shift_range=10.)

In [ ]:
valtest_datagen = ImageDataGenerator(rescale=1. / 255)

###### Visualizing images after image augmentation

###### Fit a model

In [ ]:
X_train_array.shape

###### While fitting the model, we need to reshape the training data set to (128,128,1). Here, 1 represents a grayscale image.

In [ ]:
## training data geenrator
train_generator = train_datagen.flow(X_train_array.reshape(nb_train_samples,img_height,img_width,1),
                                     y_train, batch_size=batch_size)

In [ ]:
## testing data geenrator
test_generator = valtest_datagen.flow(X_test_array.reshape(nb_validation_samples,img_height,img_width,1),
                                      y_test, batch_size=batch_size)

In [ ]:
## validation data generator
validation_generator = valtest_datagen.flow(X_val_array.reshape(nb_validation_samples,img_height,img_width,1),
                                            y_val, batch_size=batch_size)

In [ ]:
## Fit a CNN model
simple_cnn = cnn_model.fit_generator(
    train_generator, 
    steps_per_epoch=nb_train_samples/batch_size,
    epochs=3,
    validation_data=validation_generator,
    validation_steps=nb_validation_samples/batch_size
)

###### Making Predictions

In [ ]:
## Evaluating the model in the test set
simple_cnn_loss, simple_cnn_acc = cnn_model.evaluate(X_test_array.
                                                     reshape(nb_validation_samples,img_height,img_width,1),
                                                     y_test)
#test_loss, test_acc = evaluate(X_test_array, y_test)

In [ ]:
print('Test accuracy:', test_acc)

In [ ]:
# calculate predictions
predictions = cnn_model.predict(X_test_array.
                               reshape(nb_validation_samples,img_height,img_width,1))
predictions

In [ ]:
def get_rounded_predictions(predictions, threshold):
    rounded_predictions = []
    for x in predictions:
        if x[0] > threshold:
            rounded_predictions.append(1)
        else:
            rounded_predictions.append(0)
    return rounded_predictions

In [ ]:
# round predictions
rounded_predictions = get_rounded_predictions(predictions, 0.51)

#rounded_predictions = [round(x[0]) for x in predictions]
print(rounded_predictions)

In [ ]:
rounded_predictions = np.array(rounded_predictions)

In [ ]:
rounded_predictions.shape

In [ ]:
y_test.shape

#### Evaluate the simple cnn model

###### Confusion matrix

In [ ]:
# compute the confusion matrix
confusion_mtx = confusion_matrix(y_test, rounded_predictions) 

In [ ]:
## Plot the confusion matrix
plt.figure()
plot_confusion_matrix(confusion_mtx, classes)
plt.xticks(range(2), ['Normal', 'Pneumonia'], fontsize=16)
plt.yticks(range(2), ['Normal', 'Pneumonia'], fontsize=16)
plt.show()

###### Classification Report

In [ ]:
## Print classification report
print(classification_report(y_test, rounded_predictions, 
                            target_names = classes))

###### AUC/ROC curve

In [ ]:
# calculate AUC
auc = roc_auc_score(y_test, rounded_predictions)
print('AUC: %.3f' % auc)

In [ ]:
# calculate roc curve
fpr, tpr, thresholds = roc_curve(y_test, rounded_predictions)

In [ ]:
# plot random line in the middle
fig, ax1 = plt.subplots(1,1, figsize = (5, 5), dpi = 250)
plt.plot([0, 1], [0, 1], linestyle='--')
# plot the roc curve for the model
plt.plot(fpr, tpr, marker='.',
        label = 'Simple CNN Model (AUC:%.3f)' % auc)
# show the plot
ax1.legend(loc = 4)
plt.title('Receiver operating characteristic')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.show()

## Transfer Learning

### Using VGG-16 model

In [8]:
def proc_images():
    """
    Returns two arrays: 
        x is an array of resized images
        y is an array of labels
    """

    x = [] # images as arrays
    y = [] # labels 
    WIDTH = 224
    HEIGHT = 224

    #for image in enumerate(merged_df.patientId.head(5000)):
    for image in enumerate(merged_df.patientId):    
        
        img_name = image[1]
        image_path = path + '/' + img_name + '.png'

        # Read and resize image
        full_size_image = cv2.imread(image_path)
        x.append(cv2.resize(full_size_image, (WIDTH,HEIGHT), interpolation=cv2.INTER_CUBIC))
        
        # Labels
        index_of_image = image[0]
        target_value = merged_df.target.loc[index_of_image]
        #print(target_value)
        y.append(target_value)

    return x,y

In [9]:
X,y = proc_images()

In [ ]:
## Saving arrays for future use
np.savez_compressed("./data_as_arrays/x_images_224", X)
np.savez_compressed("./data_as_arrays/y_pneumonia_224", y)

In [ ]:
# First split the data in two sets, 80% for training, 20% for Val/Test)
X_train, X_test_val, y_train, y_test_val = train_test_split(X,y, test_size=0.2, random_state=1, stratify=y)

# Second split the 20% into validation and test sets
X_test, X_val, y_test, y_val = train_test_split(X_test_val, y_test_val, test_size=0.5, random_state=1, 
                                                stratify=y_test_val)

#### Normalize the dataset

In [ ]:
X_train_array = np.array(X_train)
X_test_array = np.array(X_test)
X_val_array = np.array(X_val)

In [ ]:
## Reshape the input arrays such that the values are between 0 and 1
X_train_array = X_train_array/255.0
X_test_array = X_test_array/255.0
X_val_array = X_val_array/255.0

In [ ]:
y_train = np.array(y_train)
y_test = np.array(y_test)
y_val = np.array(y_val)

In [ ]:
## Define static variables

## Image size after resizing
img_width, img_height = 224, 224

## no of training samples
nb_train_samples = len(X_train)

## no of validation samples
nb_validation_samples = len(X_val)

## No of epochs/iterations
epochs = 10

## batch size in each epoch
batch_size = 16

In [ ]:
from keras.applications.vgg16 import VGG16
from keras.preprocessing import image
from keras.models import Model
from keras.layers import Dense, GlobalAveragePooling2D
from keras import backend as K

In [ ]:
## Hyperparameters

# params we will probably want to do some hyperparameter optimization later
BASE_MODEL= 'VGG16' # ['VGG16', 'RESNET52', 'InceptionV3', 'Xception', 'DenseNet169', 'DenseNet121']
IMG_SIZE = (224, 224) # [(224, 224), (384, 384), (512, 512), (640, 640)]
BATCH_SIZE = 24 # [1, 8, 16, 24]
DENSE_COUNT = 128 # [32, 64, 128, 256]
DROPOUT = 0.25 # [0, 0.25, 0.5]
LEARN_RATE = 1e-4 # [1e-4, 1e-3, 4e-3]
TRAIN_SAMPLES = 8000 # [3000, 6000, 15000]
TEST_SAMPLES = 800
USE_ATTN = False # [True, False]

In [ ]:
## create the base pre-trained model
vgg16_model = VGG16()

In [ ]:
## Print the summary statistics to check the structure of the network
vgg16_model.summary()

In [ ]:
## Check the type of model
type(vgg16_model)

In [ ]:
from keras import models

In [ ]:
## Convert into Sequential model since our model is sequential
model = models.Sequential()

In [ ]:
for layer in vgg16_model.layers[:-1]:
    print(layer)
    model.add(layer)

In [ ]:
## Print the summary
model.summary()

###### Note:
We can see that there is no difference in the layers except for the last layer.
Now, checking at the network, we can see that there are 1000 classes in the predictions layers. But since, we need only need 1 output. So, we need to change it which is why we did not include it in our new model.

In [ ]:
## Make the layers static
for layer in model.layers:
    layer.trainable = False

In [ ]:
## Add our own prediction layer
model.add(Dense(1, activation='sigmoid'))

#### Train the VGG16 model in our dataset

In [ ]:
## compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics = ['accuracy'])

In [ ]:
## Image Augmentation
train_datagen = ImageDataGenerator(rescale=1. / 255, horizontal_flip=True, rotation_range=30)
valtest_datagen = ImageDataGenerator(rescale=1. / 255)

In [ ]:
## Generte training data
train_generator = train_datagen.flow(np.array(X_train).
                                     reshape(nb_train_samples,img_height,img_width,3),
                                     y_train, batch_size=batch_size)

In [ ]:
## Generate Validation dataset
validation_generator = valtest_datagen.flow(np.array(X_val).
                                            reshape(nb_validation_samples,img_height,img_width,3),
                                            y_val, batch_size=batch_size)

In [ ]:
## Generate test dataset
test_generator = valtest_datagen.flow(np.array(X_test).
                                      reshape(nb_validation_samples,img_height,img_width,3),
                                      y_test, batch_size=batch_size)

In [ ]:
## Fit the model
model.fit_generator(
    train_generator,
    steps_per_epoch = 5,
    #steps_per_epoch=nb_train_samples // batch_size,
    epochs=3,
    validation_data=validation_generator,
    #validation_steps=nb_validation_samples // batch_size
    validation_steps = 2
)

In [ ]:
X_test_array.shape

In [ ]:
## Evaluating the model in the test set
vgg16_loss, vgg16_acc = model.evaluate(X_test_array,y_test)
#test_loss, test_acc = evaluate(X_test_array, y_test)

In [ ]:
print('Test accuracy:', test_acc)

In [ ]:
# calculate predictions
predictions = model.predict(X_test_array.
                               reshape(nb_validation_samples,img_height,img_width,3))
predictions

In [ ]:
## round predictions
rounded_predictions = get_rounded_predictions(predictions, 0.5)

#rounded_predictions = [round(x[0]) for x in predictions]
print(rounded_predictions)

In [ ]:
## Convert the list into array
rounded_predictions = np.array(rounded_predictions)

In [ ]:
## Print the shape to confirm it 
rounded_predictions.shape

In [ ]:
## Print the shape of prediction values in test set
y_test.shape

#### Evaluate the simple cnn model

###### Confusion matrix

In [ ]:
# compute the confusion matrix
confusion_mtx = confusion_matrix(y_test, rounded_predictions) 

In [ ]:
## Plot the confusion matrix
plt.figure()
plot_confusion_matrix(confusion_mtx, classes)
plt.xticks(range(2), ['Normal', 'Pneumonia'], fontsize=16)
plt.yticks(range(2), ['Normal', 'Pneumonia'], fontsize=16)
plt.show()

###### Classification Report

In [ ]:
## Print classification report
print(classification_report(y_test, rounded_predictions, 
                            target_names = classes))

###### AUC/ROC curve

In [ ]:
# calculate AUC
auc = roc_auc_score(y_test, rounded_predictions)
print('AUC: %.3f' % auc)

In [ ]:
# calculate roc curve
fpr, tpr, thresholds = roc_curve(y_test, rounded_predictions)

In [ ]:
# plot random line in the middle
fig, ax1 = plt.subplots(1,1, figsize = (5, 5), dpi = 250)
plt.plot([0, 1], [0, 1], linestyle='--')
# plot the roc curve for the model
plt.plot(fpr, tpr, marker='.',
        label = 'Simple CNN Model (AUC:%.3f)' % auc)
# show the plot
ax1.legend(loc = 4)
plt.title('Receiver operating characteristic')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.show()